This is the notebook I used to cleanse and organise the original scraped CSV file for usage stats from 2021-2023. I will not be conducting the full EDA in this notebook, as I feel that is best suited to the notebook I use for analysis and data science application.

However, I will navigate the dataset to an extent here to understand the features, and draw conclusions on how to organise as a dataframe. The main reason this is being executed is due to the shift in data format at some given point in the usage stats, where some columns are simpy renamed to something else, giving X rows a value for one version of the given column, and Y rows having their value in the alternative version of this column.

In [2]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
#the original csv file
path = "/content/drive/MyDrive/cycling.csv"


Mounted at /content/drive


In [3]:
#reading, this can take a min or so  given the scale
df = pd.read_csv(path, low_memory=False)

In [4]:
#inspecting size of combined 2021-2023 df
df.shape

(30313809, 20)

In [5]:
#inspection of datatypes
df.dtypes

Rental Id               float64
Duration                float64
Bike Id                 float64
End Date                 object
EndStation Id           float64
EndStation Name          object
Start Date               object
StartStation Id         float64
StartStation Name        object
Number                  float64
Start date               object
Start station number     object
Start station            object
End date                 object
End station number       object
End station              object
Bike number             float64
Bike model               object
Total duration           object
Total duration (ms)     float64
dtype: object

After initial examination, it seems that the format changed for the data at some point over the period of 2021-2023.

It would be appropriate to also convert any columns into appropriate formats now, such as enddate to datetime etc. So let's start with the preprocessing.

Furthermore, it appears that there was indeed a change in format of the data, as it would not make much sense otherwise to have two variants for fields such as "Start Date" or "End Station"

In [6]:
#first, let's change the date columns into the appropriate format before combining, takes a while given size
df['Start Date'] = pd.to_datetime(df['Start Date'])
df['End Date'] = pd.to_datetime(df['End Date'])

df['Start date'] = pd.to_datetime(df['Start date'])
df['End date'] = pd.to_datetime(df['End date'])

In [7]:
#checking conversion
df.dtypes

Rental Id                      float64
Duration                       float64
Bike Id                        float64
End Date                datetime64[ns]
EndStation Id                  float64
EndStation Name                 object
Start Date              datetime64[ns]
StartStation Id                float64
StartStation Name               object
Number                         float64
Start date              datetime64[ns]
Start station number            object
Start station                   object
End date                datetime64[ns]
End station number              object
End station                     object
Bike number                    float64
Bike model                      object
Total duration                  object
Total duration (ms)            float64
dtype: object

An initial suspicion is the contrast between the "Number" column and the "Rental Id" column. At first glance, it appears later data entries reverted from using the rental id to the number id as the unique identifier of the event. I will now explore this and see if there is any weight to this assumption

In [8]:
def check_counterparts(col1,col2):

  '''
  Function for comparing column counts,
  used to examine two columns that are likely to be the same data but spread over 2 columns
  due to changes in data format

  Args:
    col1(pandas.series): The first column of interest
    col2(pandas.series): The second column of interest

  Returns:
    None

  Prints:
    Prints out counts of col1 and col2 in dataframe that are NaN values, as well as
    the count of non-NaN values.

  '''

  column1_count = col1.notna().sum()
  column2_count = col2.notna().sum()

  column1_NaN_count = col1.isna().sum()
  column2_NaN_count = col2.isna().sum()



  print(f'Rows with a column 1 entry: {column1_count}\n')
  print(f'Rows with a column 2 entry: {column2_count}\n')

  print(f'Rows missing a column 1 entry: {column1_NaN_count}\n')
  print(f'Rows missing a column 2 entry: {column2_NaN_count}\n')


  return None

In [9]:
#let's examine the number and rental id columns
check_counterparts(df['Number'], df['Rental Id'])

Rows with a column 1 entry: 10586532

Rows with a column 2 entry: 19727277

Rows missing a column 1 entry: 19727277

Rows missing a column 2 entry: 10586532



As observed, it appears this suspicion is correct. It now makes sense to rectify this by unifying both columns into a singular "Number ID" column to avoid difficulties later.

In [10]:
#creating new number id column, using rental id and number
df['Number ID'] = df['Rental Id'].combine_first(df['Number'])

#checking we have a value for each row to confirm
df['Number ID'].shape[0] == df.shape[0]

True

In [11]:
#now we can drop rental id and number
df.drop(['Rental Id', 'Number'], axis=1, inplace=True)

In [12]:
df.dtypes

Duration                       float64
Bike Id                        float64
End Date                datetime64[ns]
EndStation Id                  float64
EndStation Name                 object
Start Date              datetime64[ns]
StartStation Id                float64
StartStation Name               object
Start date              datetime64[ns]
Start station number            object
Start station                   object
End date                datetime64[ns]
End station number              object
End station                     object
Bike number                    float64
Bike model                      object
Total duration                  object
Total duration (ms)            float64
Number ID                      float64
dtype: object

Next, something should be done about the conflicting start and end dates, maybe we combine them in similar fashion to the rental id and number:

In [13]:
print(check_counterparts(df['Start Date'], df['Start date']))
print(check_counterparts(df['End Date'], df['End date']))

Rows with a column 1 entry: 19727277

Rows with a column 2 entry: 10586532

Rows missing a column 1 entry: 10586532

Rows missing a column 2 entry: 19727277

None
Rows with a column 1 entry: 19727277

Rows with a column 2 entry: 10586532

Rows missing a column 1 entry: 10586532

Rows missing a column 2 entry: 19727277

None


Again, seems like it is the case with these dates that some rows changed to a different format similar to the number and rental id dilemma. Time to convert in similar fashion into a singular column, and drop the two redundant columns as a result

In [15]:
#creating new number id column, using rental id and number
df['Start'] = df['Start Date'].combine_first(df['Start date'])
df['End'] = df['End Date'].combine_first(df['End date'])


#checking we have a value for each row to confirm
df['Start'].shape[0] == df.shape[0] == df['End'].shape[0]

True

In [16]:
#now we can drop rental id and number
df.drop(['Start Date', 'End Date', 'Start date', 'End date'], axis=1, inplace=True)

In [17]:
df.dtypes

Duration                       float64
Bike Id                        float64
EndStation Id                  float64
EndStation Name                 object
StartStation Id                float64
StartStation Name               object
Start station number            object
Start station                   object
End station number              object
End station                     object
Bike number                    float64
Bike model                      object
Total duration                  object
Total duration (ms)            float64
Number ID                      float64
Start                   datetime64[ns]
End                     datetime64[ns]
dtype: object

It also seems that there are a few more intertwined fields:


*   EndStation Name and End station
*   EndStation Id and End station number

*   StartStation Name and Start station
*   StartStation Id and Start station number

We should do similar for these fields, then it appears there are no more conflicts, but we will review the columns further to ensure.





In [23]:
check_counterparts(df['EndStation Name'],df['End station'])

Rows with a column 1 entry: 19727277

Rows with a column 2 entry: 10586532

Rows missing a column 1 entry: 10586532

Rows missing a column 2 entry: 19727277



In [25]:
check_counterparts(df['EndStation Id'], df['End station number'])

Rows with a column 1 entry: 19415133

Rows with a column 2 entry: 10586532

Rows missing a column 1 entry: 10898676

Rows missing a column 2 entry: 19727277



In [24]:
check_counterparts(df['StartStation Name'], df['Start station'])

Rows with a column 1 entry: 19727277

Rows with a column 2 entry: 10586532

Rows missing a column 1 entry: 10586532

Rows missing a column 2 entry: 19727277



In [26]:
check_counterparts(df['StartStation Id'], df['Start station number'])

Rows with a column 1 entry: 19727277

Rows with a column 2 entry: 10586532

Rows missing a column 1 entry: 10586532

Rows missing a column 2 entry: 19727277



In [35]:
check_counterparts(df['StartStation Name'], df['Start station'])

Rows with a column 1 entry: 19727277

Rows with a column 2 entry: 10586532

Rows missing a column 1 entry: 10586532

Rows missing a column 2 entry: 19727277



In [36]:
check_counterparts(df['EndStation Name'], df['End station'])

Rows with a column 1 entry: 19727277

Rows with a column 2 entry: 10586532

Rows missing a column 1 entry: 10586532

Rows missing a column 2 entry: 19727277



Suspicions largely confirmed, it is important to note after inspecting dtypes earlier:

1.   The Start/End station Id columns use float64, but the Start/End station number columns use an object datatype, this can be ammended.
2.   It appears Endstation Id is missing some entries, this will be explored further in EDA and model application, but one immediate thought is that the journeys failed to be logged at destination for numerous reasons.




In [28]:
#transforming to numeric, using .to_numeric to handle any problematic rows
df['Start station number'] = pd.to_numeric(df['Start station number'], errors='coerce')
df['End station number'] = pd.to_numeric(df['End station number'], errors='coerce')

#creating new number id column
df['Start station code'] = df['StartStation Id'].combine_first(df['Start station number'])
df['End station code'] = df['EndStation Id'].combine_first(df['End station number'])

#creating new station name column
df['Start Station'] = df['StartStation Name'].combine_first(df['Start station'])
df['End Station'] = df['EndStation Name'].combine_first(df['End station'])

In [33]:
#dropping old columns
df.drop(['StartStation Id',
         'EndStation Id',
         'Start station number',
         'End station number',
         'StartStation Name',
         'EndStation Name',
         'Start station',
         'End station'],
         axis=1,
         inplace=True)

In [40]:
#final check of datatypes
df.dtypes

Duration                      float64
Bike Id                       float64
Bike number                   float64
Bike model                     object
Total duration                 object
Total duration (ms)           float64
Number ID                     float64
Start                  datetime64[ns]
End                    datetime64[ns]
Start station code            float64
End station code              float64
Start Station                  object
End Station                    object
dtype: object

The dataframe has now been preprocessed appropriately, ready for EDA and feature engineering in a separate notebook.

In [42]:
#saving to csv
df.to_csv('/content/drive/MyDrive/processed_cycling.csv', index=False)